In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

In [4]:
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=6, shuffle=True)

# Download and load the test data
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=6, shuffle=True)

In [5]:
# Define the network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2) # Input channel is 1, output channels is 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024) # Fully connected layers
        self.fc2 = nn.Linear(1024, 10) # Output 10 classes for digits 0-9

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # Max pooling over a (2, 2) window
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 7*7*64) # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
net = Net().to(device)

In [6]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
# Train the network
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.472
[1,  4000] loss: 0.112
[1,  6000] loss: 0.084
[1,  8000] loss: 0.066
[1, 10000] loss: 0.059
[2,  2000] loss: 0.047
[2,  4000] loss: 0.043
[2,  6000] loss: 0.047
[2,  8000] loss: 0.041
[2, 10000] loss: 0.038
[3,  2000] loss: 0.032
[3,  4000] loss: 0.028
[3,  6000] loss: 0.029
[3,  8000] loss: 0.028
[3, 10000] loss: 0.028
[4,  2000] loss: 0.019
[4,  4000] loss: 0.021
[4,  6000] loss: 0.021
[4,  8000] loss: 0.023
[4, 10000] loss: 0.024
[5,  2000] loss: 0.016
[5,  4000] loss: 0.015
[5,  6000] loss: 0.017
[5,  8000] loss: 0.015
[5, 10000] loss: 0.016
[6,  2000] loss: 0.010
[6,  4000] loss: 0.012
[6,  6000] loss: 0.011
[6,  8000] loss: 0.014
[6, 10000] loss: 0.015
[7,  2000] loss: 0.007
[7,  4000] loss: 0.009
[7,  6000] loss: 0.009
[7,  8000] loss: 0.010
[7, 10000] loss: 0.010
[8,  2000] loss: 0.006
[8,  4000] loss: 0.009
[8,  6000] loss: 0.007
[8,  8000] loss: 0.008
[8, 10000] loss: 0.008
[9,  2000] loss: 0.004
[9,  4000] loss: 0.006
[9,  6000] loss: 0.008
[9,  8000] 

In [9]:
# Test the network
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # calculate outputs by running images through the network
        outputs = net(images)
        
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total):.2f} %')

Accuracy of the network on the 10000 test images: 99.25 %
